In [18]:
# Let's solve the composite-wall problem using the provided solver,
# extending it slightly to support convective (Robin) boundary conditions.

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable, Tuple, Literal

BCType = Literal["Dirichlet", "Neumann", "Robin"]  # Robin = convection: (-k dT/dx) = h (T_surface - T_inf)

@dataclass
class MaterialFuncs:
    rho: Callable[[np.ndarray, float, np.ndarray], np.ndarray]  # rho(x, t, T)
    cp: Callable[[np.ndarray, float, np.ndarray], np.ndarray]   # cp(x, t, T)
    k:  Callable[[np.ndarray, float, np.ndarray], np.ndarray]   # k(x, t, T)

@dataclass
class SourceFunc:
    qdot: Callable[[np.ndarray, float], np.ndarray]  # qdot(x, t) [W/m^3]

@dataclass
class BCFunc:
    # For "Robin", return value as a 2-tuple: (h, T_inf)
    left:  Callable[[float], Tuple[BCType, object]]
    right: Callable[[float], Tuple[BCType, object]]

@dataclass
class Grid1D:
    L: float
    N: int

    def __post_init__(self):
        assert self.N >= 2, "Need at least 2 grid points"
        self.x = np.linspace(0.0, self.L, self.N)
        self.dx = self.x[1] - self.x[0]

def harmonic_mean(a, b, eps=1e-20):
    """Harmonic mean guard against zeros."""
    return 2.0 * a * b / np.maximum(a + b, eps)

def thomas_solve(a, b, c, d):
    """
    Solve tridiagonal system with subdiag a[1:], diag b, superdiag c[:-1], RHS d.
    All arrays length N. In-place safe for copies created here.
    """
    n = len(b)
    ac, bc, cc, dc = map(np.array, (a.copy(), b.copy(), c.copy(), d.copy()))
    # Forward elimination
    for i in range(1, n):
        m = ac[i] / bc[i-1]
        bc[i] -= m * cc[i-1]
        dc[i] -= m * dc[i-1]
    # Back substitution
    x = np.zeros_like(dc)
    x[-1] = dc[-1] / bc[-1]
    for i in range(n-2, -1, -1):
        x[i] = (dc[i] - cc[i] * x[i+1]) / bc[i]
    return x

class Heat1DSolver:
    """
    Generalized 1D heat equation solver with variable properties and time-dependent BCs.

    Governing equation:
        rho(x,t,T) * cp(x,t,T) * dT/dt = d/dx[ k(x,t,T) * dT/dx ] + qdot(x,t)

    Discretization:
        - Uniform grid, finite-volume / finite-difference hybrid
        - Crank–Nicolson (theta-method) in time with optional Picard iterations for nonlinearity
        - Dirichlet BC: T at boundary node set directly
        - Neumann BC: specified heat flux q''(t) [W/m^2], positive INTO domain
        - Robin BC: convection: -k dT/dx = h (T_surface - T_inf(t))
    """
    def __init__(
        self,
        grid: Grid1D,
        material: MaterialFuncs,
        source: SourceFunc,
        bc: BCFunc,
        theta: float = 0.5,         # 0.5 = Crank–Nicolson, 1 = implicit Euler
        max_iter: int = 10,
        tol: float = 1e-8,
    ):
        assert 0.0 < theta <= 1.0, "theta in (0,1]"
        self.g = grid
        self.mat = material
        self.src = source
        self.bc = bc
        self.theta = theta
        self.max_iter = max_iter
        self.tol = tol

    def _assemble_system(self, Tn: np.ndarray, dt: float, t_n: float, t_np1: float):
        """
        Assemble tridiagonal system a,b,c,d for solve at time t^{n+1}.
        We use Picard linearization: properties evaluated at (x, t*, T*), where t* = t^{n+1} for implicit part,
        and at (x, t^n, T^n) for explicit part.
        """
        x, dx = self.g.x, self.g.dx
        N = self.g.N
        theta = self.theta

        # Properties at old time (explicit) and predictors at new time (implicit)
        rho_n = self.mat.rho(x, t_n, Tn)
        cp_n  = self.mat.cp(x, t_n, Tn)
        k_n   = self.mat.k(x, t_n, Tn)

        # Begin with a predictor for T^{n+1} to evaluate props; use Tn initially
        Tstar = Tn.copy()

        # Initialize tridiagonal arrays
        a = np.zeros(N)  # sub-diagonal
        b = np.zeros(N)  # diagonal
        c = np.zeros(N)  # super-diagonal
        d = np.zeros(N)  # RHS

        # Precompute explicit operator L(Tn) and qdot parts
        k_e_n = harmonic_mean(k_n[:-1], k_n[1:])  # faces i+1/2, length N-1
        L_Tn = np.zeros(N)
        L_Tn[1:-1] = (k_e_n[1:] * (Tn[2:] - Tn[1:-1]) - k_e_n[:-1] * (Tn[1:-1] - Tn[:-2])) / (dx*dx)
        q_n = self.src.qdot(x, t_n)

        # Picard iterations
        for _it in range(self.max_iter):
            rho_np1 = self.mat.rho(x, t_np1, Tstar)
            cp_np1  = self.mat.cp(x, t_np1, Tstar)
            k_np1   = self.mat.k(x, t_np1, Tstar)

            Cn    = rho_n * cp_n
            Cnp1  = rho_np1 * cp_np1

            # Implicit face conductivities
            k_e_np1 = harmonic_mean(k_np1[:-1], k_np1[1:])

            # Reset a,b,c,d
            a[:] = 0.0; b[:] = 0.0; c[:] = 0.0; d[:] = 0.0

            # Interior nodes
            for i in range(1, N-1):
                Ai = theta * dt / Cnp1[i] * k_e_np1[i-1] / (dx*dx)
                Ci = theta * dt / Cnp1[i] * k_e_np1[i]   / (dx*dx)
                Bi = 1.0 + Ai + Ci

                a[i] = -Ai
                b[i] = Bi
                c[i] = -Ci

                # RHS from previous time and explicit part
                explicit_diff = L_Tn[i] / Cn[i]
                q_term = (theta * dt * self.src.qdot(np.array([x[i]]), t_np1)[0] / Cnp1[i]
                          + (1 - theta) * dt * q_n[i] / Cn[i])

                d[i] = Tn[i] + dt * ( (1 - theta) * explicit_diff ) + q_term

            # Boundary conditions
            # LEFT boundary (i=0)
            bcL_type, bcL_val_np1 = self.bc.left(t_np1)
            if bcL_type == "Dirichlet":
                a[0] = 0.0; b[0] = 1.0; c[0] = 0.0; d[0] = bcL_val_np1
            elif bcL_type == "Neumann":
                Ci = theta * dt / ( (rho_np1[0]*cp_np1[0]) ) * (k_e_np1[0]) / (dx*dx)
                b[0] = 1.0 + Ci
                c[0] = -Ci
                a[0] = 0.0
                explicit_diff_0 = (k_e_n[0] * (Tn[1] - Tn[0])) / (dx*dx) / (rho_n[0]*cp_n[0])
                q_term0 = (theta * dt * self.src.qdot(np.array([x[0]]), t_np1)[0] / (rho_np1[0]*cp_np1[0])
                           + (1 - theta) * dt * q_n[0] / (rho_n[0]*cp_n[0]))
                qL_np1 = bcL_val_np1  # W/m^2, positive into domain
                d[0] = Tn[0] + dt * (1 - theta) * explicit_diff_0 + q_term0 + theta * dt * (2.0 * qL_np1 / (rho_np1[0]*cp_np1[0] * dx))
            elif bcL_type == "Robin":
                # convection: -k dT/dx = h (T0 - T_inf)
                hL, Tinf_np1 = bcL_val_np1
                _, (hL_n, Tinf_n) = self.bc.left(t_n)
                Ci = theta * dt / (rho_np1[0]*cp_np1[0]) * k_e_np1[0] / (dx*dx)
                b[0] = 1.0 + Ci + theta * dt / (rho_np1[0]*cp_np1[0]) * (2.0 * hL / dx)
                c[0] = -Ci
                a[0] = 0.0
                explicit_diff_0 = (
                    (k_e_n[0] * (Tn[1] - Tn[0])) / (dx*dx)
                    + (2.0 * hL_n / dx) * (Tinf_n - Tn[0])
                ) / (rho_n[0]*cp_n[0])
                q_term0 = (theta * dt * self.src.qdot(np.array([x[0]]), t_np1)[0] / (rho_np1[0]*cp_np1[0])
                           + (1 - theta) * dt * q_n[0] / (rho_n[0]*cp_n[0]))
                # RHS includes implicit part from T_inf at n+1
                d[0] = Tn[0] + dt * (1 - theta) * explicit_diff_0 + q_term0 + theta * dt / (rho_np1[0]*cp_np1[0]) * (2.0 * hL / dx) * Tinf_np1
            else:
                raise ValueError("Unknown BC type on left")

            # RIGHT boundary (i=N-1)
            bcR_type, bcR_val_np1 = self.bc.right(t_np1)
            if bcR_type == "Dirichlet":
                a[N-1] = 0.0; b[N-1] = 1.0; c[N-1] = 0.0; d[N-1] = bcR_val_np1
            elif bcR_type == "Neumann":
                Ai = theta * dt / ( (rho_np1[-1]*cp_np1[-1]) ) * (k_e_np1[-1]) / (dx*dx)
                a[N-1] = -Ai
                b[N-1] = 1.0 + Ai
                c[N-1] = 0.0
                explicit_diff_N = (- k_e_n[-1] * (Tn[-1] - Tn[-2])) / (dx*dx) / (rho_n[-1]*cp_n[-1])
                q_termN = (theta * dt * self.src.qdot(np.array([x[-1]]), t_np1)[0] / (rho_np1[-1]*cp_np1[-1])
                           + (1 - theta) * dt * q_n[-1] / (rho_n[-1]*cp_n[-1]))
                qR_np1 = bcR_val_np1  # positive into domain
                d[N-1] = Tn[-1] + dt * (1 - theta) * explicit_diff_N + q_termN + theta * dt * (2.0 * qR_np1 / (rho_np1[-1]*cp_np1[-1] * dx))
            elif bcR_type == "Robin":
                hR, Tinf_np1 = bcR_val_np1
                _, (hR_n, Tinf_n) = self.bc.right(t_n)
                Ai = theta * dt / (rho_np1[-1]*cp_np1[-1]) * k_e_np1[-1] / (dx*dx)
                a[N-1] = -Ai
                b[N-1] = 1.0 + Ai + theta * dt / (rho_np1[-1]*cp_np1[-1]) * (2.0 * hR / dx)
                c[N-1] = 0.0
                explicit_diff_N = (
                    (- k_e_n[-1] * (Tn[-1] - Tn[-2])) / (dx*dx)
                    + (2.0 * hR_n / dx) * (Tinf_n - Tn[-1])
                ) / (rho_n[-1]*cp_n[-1])
                q_termN = (theta * dt * self.src.qdot(np.array([x[-1]]), t_np1)[0] / (rho_np1[-1]*cp_np1[-1])
                           + (1 - theta) * dt * q_n[-1] / (rho_n[-1]*cp_n[-1]))
                d[N-1] = Tn[-1] + dt * (1 - theta) * explicit_diff_N + q_termN + theta * dt / (rho_np1[-1]*cp_np1[-1]) * (2.0 * hR / dx) * Tinf_np1
            else:
                raise ValueError("Unknown BC type on right")

            # Solve tridiagonal
            Tnew = thomas_solve(a, b, c, d)

            # Check convergence
            if np.linalg.norm(Tnew - Tstar, ord=np.inf) < self.tol:
                return a, b, c, d, Tnew
            Tstar = Tnew

        # If not converged, still return last assembled and Tstar
        return a, b, c, d, Tstar

    def step(self, Tn: np.ndarray, dt: float, t_n: float):
        """Advance one time step using Picard iterations inside implicit solve."""
        t_np1 = t_n + dt
        a, b, c, d, Tnp1 = self._assemble_system(Tn, dt, t_n, t_np1)
        return Tnp1, t_np1

    def run(self, T0: np.ndarray, t0: float, tf: float, dt: float, callback=None):
        """Run from t0 to tf, invoking callback(t, T) each step if provided."""
        T = T0.copy()
        t = t0
        while t < tf - 1e-15:
            dt_eff = min(dt, tf - t)
            T, t = self.step(T, dt_eff, t)
            if callback is not None:
                callback(t, T)
        return T, t



In [ ]:
# -----------------------
# Problem set-up

# Geometry (m)
L_plaster = 0.010   # 10 mm
L_fibreg  = 0.100   # 100 mm
L_plywood = 0.020   # 20 mm
L_total   = L_plaster + L_fibreg + L_plywood

# Grid
N = 350  # fine grid for smooth interfaces
grid = Grid1D(L=L_total, N=N)

# Material properties (assumed typical values)
# (If your textbook problem 8 lists different values, swap them here.)
k_plaster = 0.17   # W/m-K (gypsum board)
rho_plaster = 950.0  # kg/m3
cp_plaster = 840.0   # J/kg-K

k_fibreg = 0.040   # W/m-K
rho_fibreg = 12.0  # kg/m3
cp_fibreg = 800.0  # J/kg-K

k_plywood = 0.12   # W/m-K
rho_plywood = 540.0  # kg/m3
cp_plywood = 1600.0  # J/kg-K

# Piecewise-constant property helpers
x_break1 = L_plaster
x_break2 = L_plaster + L_fibreg

def pick_region(x):
    """Return 0 for plaster, 1 for fibreglass, 2 for plywood, vectorized."""
    reg = np.zeros_like(x, dtype=int)
    reg[x >= x_break1] = 1
    reg[x >= x_break2] = 2
    return reg

def rho_fun(x, t, T):
    reg = pick_region(x)
    out = np.empty_like(x)
    out[reg==0] = rho_plaster
    out[reg==1] = rho_fibreg
    out[reg==2] = rho_plywood
    return out

def cp_fun(x, t, T):
    reg = pick_region(x)
    out = np.empty_like(x)
    out[reg==0] = cp_plaster
    out[reg==1] = cp_fibreg
    out[reg==2] = cp_plywood
    return out

def k_fun(x, t, T):
    reg = pick_region(x)
    out = np.empty_like(x)
    out[reg==0] = k_plaster
    out[reg==1] = k_fibreg
    out[reg==2] = k_plywood
    return out

material = MaterialFuncs(rho=rho_fun, cp=cp_fun, k=k_fun)

# No internal heat generation
source = SourceFunc(qdot=lambda x, t: np.zeros_like(x))

# Convection data
hi = 15.0  # W/m2-K (inside)
ho = 60.0  # W/m2-K (outside)
Ti = 293.0 # K (room air, constant)

# Outside air temperature To(t), t in seconds -> convert to hours for the given expression
def To_time(t):
    hours = t / 3600.0
    if hours <= 12.0:
        return 273.0 + 5.0 * np.sin(np.pi * hours / 12.0)
    else:
        return 273.0 + 30.0 * np.sin(np.pi * hours / 12.0)

# Boundary condition functions (Robin: convection)
def left_bc(t):
    # left is the room side (plasterboard exposed to room air)
    return ("Robin", (hi, Ti))

def right_bc(t):
    # right is outdoors (plywood exposed to outside air)
    return ("Robin", (ho, To_time(t)))

bc = BCFunc(left=left_bc, right=right_bc)

# Solver
solver = Heat1DSolver(grid=grid, material=material, source=source, bc=bc, theta=0.5)

# Initial condition (start at indoor temperature everywhere)
T_init = np.full(grid.N, Ti)

# Time integration: run for several days to reach periodic steady state
day = 24 * 3600.0
t_total = 8 * day       # run 8 days
dt = 60.0               # 1-minute steps

times = []
T_record = []

# We'll also track interface temperatures and heat fluxes
x = grid.x
i_int1 = np.argmin(np.abs(x - x_break1))  # plaster/fibreglass interface
i_int2 = np.argmin(np.abs(x - x_break2))  # fibreglass/plywood interface
T_int1 = []
T_int2 = []
q_out = []  # outward heat flux on the external surface
q_in = []   # heat leaving the room (positive into wall)

def callback(t, T):
    times.append(t)
    T_record.append(T.copy())
    T_int1.append(T[i_int1])
    T_int2.append(T[i_int2])
    # Convective heat fluxes (+) defined as "from solid to fluid" (out of the solid)
    To = To_time(t)
    q_out.append(ho * (T[-1] - To))  # W/m2, outward to ambient
    q_in.append(hi * (Ti - T[0]))    # W/m2, leaving room into wall

# Run
_ = solver.run(T0=T_init, t0=0.0, tf=t_total, dt=dt, callback=callback)

times = np.array(times)
T_int1 = np.array(T_int1)
T_int2 = np.array(T_int2)
q_out = np.array(q_out)
q_in = np.array(q_in)

# Consider only the last 24 hours (periodic steady state)
mask_last_day = times >= (t_total - day)
t_last = times[mask_last_day] - times[mask_last_day][0]
Tint1_last = T_int1[mask_last_day]
Tint2_last = T_int2[mask_last_day]
qout_last  = q_out[mask_last_day]
qin_last   = q_in[mask_last_day]

# Time-averaged heat loss to outside (as W/m^2). We'll report both sides for a check.
qout_mean = qout_last.mean()
qin_mean  = qin_last.mean()
E_day_per_area_out = qout_last.mean() * day  # J/m2 over 24 h

# Also compute the simple steady (resistance) prediction for sanity check:
R_total = (1/hi) + (L_plaster/k_plaster) + (L_fibreg/k_fibreg) + (L_plywood/k_plywood) + (1/ho)
U = 1.0 / R_total
DeltaT_mean = Ti - (273.0)  # mean outside air is 273 K because <sin> = 0
q_mean_resistance = U * DeltaT_mean

# Print key numbers
print("=== Key results (last 24 hours in periodic steady state) ===")
print(f"Average heat loss to outside (via outside film): {qout_mean:.3f} W/m^2")
print(f"Average heat leaving the room (via inside film): {qin_mean:.3f} W/m^2")
print(f"Daily energy lost per unit area (outside): {E_day_per_area_out/1000:.2f} kJ/m^2 per day")
print("Sanity check (overall U * mean ΔT):")
print(f"  Overall R = {R_total:.3f} m^2·K/W -> U = {U:.3f} W/m^2·K, ΔT_mean = {DeltaT_mean:.1f} K")
print(f"  Predicted average by resistance method: {q_mean_resistance:.3f} W/m^2")

# ---- Plots ----

# 1) Interface temperatures over the last day
plt.figure(figsize=(8,4.5))
plt.plot(t_last/3600.0, Tint1_last, label="At plaster–fibreglass interface (x=0.01 m)")
plt.plot(t_last/3600.0, Tint2_last, label="At fibreglass–plywood interface (x=0.11 m)")
plt.xlabel("Time over last 24 h [h]")
plt.ylabel("Interface temperature [K]")
plt.title("Interface temperatures (periodic steady state)")
plt.legend()
plt.tight_layout()
plt.show()

# 2) Heat fluxes through the wall (last day)
plt.figure(figsize=(8,4.2))
plt.plot(t_last/3600.0, qout_last, label="q_out (to outdoors)")
plt.plot(t_last/3600.0, qin_last, label="q_in (leaving room)")
plt.xlabel("Time over last 24 h [h]")
plt.ylabel("Heat flux [W/m²]")
plt.title("Heat loss/gain over last 24 h (periodic steady state)")
plt.legend()
plt.tight_layout()
plt.show()

# 3) Outside air temperature over the last day for reference
To_last = np.array([To_time(t) for t in times[mask_last_day]])
plt.figure(figsize=(8,3.8))
plt.plot(t_last/3600.0, To_last)
plt.xlabel("Time over last 24 h [h]")
plt.ylabel("Outside air temperature [K]")
plt.title("Outside air temperature profile (last 24 h)")
plt.tight_layout()
plt.show()


In [19]:
def solve_prob1_plastic_fusion(
    N: int = 7,
    theta: float = 0.5,
    dt: float | None = None,
    t_max: float = 3600.0,
    T_std: float = 30.0,
    T_bnd: float = 250.0,
    T_melt: float = 140.0,
):
    """
    Prob 1: Two plastic sheets, each 5 mm, pressed between 250 C plates.
    Find time until the interface (center) reaches 140 C.

    Uses the Heat1DSolver you provided (Crank–Nicolson + Picard).
    Returns (t_fuse, history, final_T, grid, material_info).
    """
    # Geometry
    b = 0.005            # each sheet 5 mm
    L = 2 * b            # total thickness
    grid = Grid1D(L=L, N=N)

    # Material (constants, but functions of (x,t,T) as required)
    k_val   = 0.26       # W/m-K
    rho_val = 1300.0     # kg/m^3
    cp_val  = 2000.0     # J/kg-K
    alpha   = k_val / (rho_val * cp_val)  # ~1e-7 m^2/s

    def rho_fn(x, t, T): return np.full_like(x, rho_val, dtype=float)
    def cp_fn(x, t, T):  return np.full_like(x, cp_val,  dtype=float)
    def k_fn(x, t, T):   return np.full_like(x, k_val,   dtype=float)

    material = MaterialFuncs(rho=rho_fn, cp=cp_fn, k=k_fn)

    # Source term (none)
    def qdot_fn(x, t): return np.zeros_like(x)
    source = SourceFunc(qdot=qdot_fn)

    # Boundary conditions: both ends held at T_bnd (Dirichlet)
    def bc_left(t):  return "Dirichlet", T_bnd
    def bc_right(t): return "Dirichlet", T_bnd
    bc = BCFunc(left=bc_left, right=bc_right)

    # Initial condition
    T = np.full(grid.N, T_std, dtype=float)

    # Choose a time step: CN has no stability limit, but use diffusive scaling for accuracy
    dx = grid.dx
    if dt is None:
        dt = 0.2 * dx * dx / alpha   # conservative for accuracy

    # Build solver
    solver = Heat1DSolver(grid, material, source, bc, theta=theta, max_iter=30, tol=1e-10)

    # Interface index (center)
    i_center = grid.N // 2

    # Time loop until fusion or t_max
    t = 0.0
    history = {"time": [], "T_center": [], "T_left": [], "T_right": []}
    while t < t_max - 1e-15:
        # advance one step
        T, t = solver.step(T, dt, t)

        # record
        history["time"].append(t)
        history["T_center"].append(T[i_center])
        history["T_left"].append(T[0])
        history["T_right"].append(T[-1])

        # fusion check
        if T[i_center] >= T_melt:
            return t, history, T, grid, {"k": k_val, "rho": rho_val, "cp": cp_val, "alpha": alpha, "dt": dt}

    # If not fused within t_max
    return None, history, T, grid, {"k": k_val, "rho": rho_val, "cp": cp_val, "alpha": alpha, "dt": dt}

t_fuse, hist, T_final, grid, info = solve_prob1_plastic_fusion()
print(f"dx = {grid.dx:.3e} m, dt = {info['dt']:.3e} s, alpha = {info['alpha']:.3e} m^2/s")
if t_fuse is None:
    print("Did not reach 140 C within t_max.")
else:
    print(f"Interface reached 140 C at t = {t_fuse:.2f} s")



dx = 1.667e-03 m, dt = 5.556e+00 s, alpha = 1.000e-07 m^2/s
Interface reached 140 C at t = 100.00 s


Fusion problem